# Loading Data and Cleaning

In [218]:
#import basic system functions
import os
import datetime

from datetime import date
from collections import Counter

#import libraries for data wrangling
import pandas as pd
import numpy as np

#import libraries for plotting data
import matplotlib.pyplot as plt


%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.plotting import autocorrelation_plot



#import libraries for time series analysis
#from fbprophet import Prophet
from pylab import rcParams
import statsmodels.api as sm
import holidays


import warnings
warnings.filterwarnings('ignore')


from IPython.display import Image
from datetime import datetime

<span style='font-family:Tahoma'>   </span>

In [219]:
df = pd.read_csv("/Users/brittarover/Capstone/Meteolytics/meteolytix_Artikelgruppen_Umsatz_verschiedeneStandorte.csv")

In [220]:
df.head()

,Filiale,Datum,Artikelgruppe,Umsatz
0,1,2015-01-07,1,50.092235
1,1,2015-01-27,1,57.002341
2,1,2015-01-30,1,52.920825
3,1,2015-02-02,1,89.343474
4,1,2015-02-05,1,68.717498



### Definitions

- **Filiale** - There are three different filial stores which are analyzed, one direct in the city (filiale=1), one near the water front (filiale=2) and the third in a residential area (filiale=3)
1 = city; 2 = waterfront area; 3 = city disctrict
- **Datum** - date of the sales data. The period 
- **Warengruppe** - main article group
1 = Brot; 2 = Brötchen; 3 = Spezial_Brötchen; 4 = Konditorei; 5 = Kuchen; 6 = Weihnachtsartikel    
- **Artikelgruppe** - article group
1 = Weizenbrot; 2 = Mischbrot; 3 = Vollkornbrot; 4 = Spezialbrot; 5 = Stangenbrote; 6 = Brötchen; 7 = Süße Brötchen; 8 = Herzhafte Brötchen; 9 = KonditoreiBlech1; 10 = KonditoreiBlech2; 11 = Stückgebäck; 12 = Blechkuchen; 13 = Weiohnachtsartikel;     
- **Umsatz** - Sales volumne per day and article group   

### Modify Dataframe with groupby and unstack 

In [221]:
df_new=pd.pivot_table(df, values='Umsatz', index=['Datum','Filiale'],columns=['Artikelgruppe'], 
                        aggfunc=np.sum, fill_value=0).reset_index()

In [222]:
#create empty dataframe with 'complete' datetime index
date_df = pd.DataFrame(index=pd.date_range(start='1/1/2015', end=df.Datum.max()))
df_new_1 = pd.pivot_table(df[df.Filiale == 1], values='Umsatz', index=['Datum'],columns=['Artikelgruppe'], 
                        aggfunc=np.sum, fill_value=0)
#fill missing days due to store closings with 0 
df_new_1=df_new_1.merge(date_df, how='right', left_index= True,right_index=True).fillna(0)
df_new_1.index = df_new_1.index.set_names(['Datum'])
df_new_1=df_new_1.reset_index()

df_new_2 = pd.pivot_table(df[df.Filiale == 2], values='Umsatz', index=['Datum'],columns=['Artikelgruppe'], 
                        aggfunc=np.sum, fill_value=0)
#fill missing days due to store closings with 0 
df_new_2 = df_new_2.merge(date_df, how='right',left_index= True,right_index=True).fillna(0)
df_new_2.index = df_new_2.index.set_names(['Datum'])
df_new_2=df_new_2.reset_index()

df_new_3 = pd.pivot_table(df[df.Filiale == 3], values='Umsatz', index=['Datum'],columns=['Artikelgruppe'], 
                        aggfunc=np.sum, fill_value=0)
#fill missing days due to store closings with 0 
df_new_3 = df_new_3.merge(date_df, how='right',left_index= True,right_index=True).fillna(0)
df_new_3.index = df_new_3.index.set_names(['Datum'])
df_new_3=df_new_3.reset_index()

In [223]:

# Renaming Artikelgruppe for all new dataframes
liste=[df_new,df_new_1,df_new_2,df_new_3]

for i in liste:
  i.rename(columns={1:'Weizenbrot',2:"Mischbrot",3:'Vollkornbrot',4:"Stangenbrote",5:"Spezialbrot",6 : "Brötchen",
                       7 : "Süsse_Brötchen",8 :"Herzhafte_Brötchen",9 : "KonditoreiBlech1",10: "KonditoreiBlech2",
                       11: "Stückgebäck", 12:"Blechkuchen",13 : "Weihnachtsartikel"
                          }, 
                 inplace=True)

### Transformation to Datetime

In [224]:
# Transformation to Datetime
dataframe=[df,df_new,df_new_1,df_new_2,df_new_3]

for i in dataframe:
  i['Datum'] = pd.to_datetime(i['Datum'])


### Additional Date columns

In [225]:
for i in dataframe:
   i['Wochentag']=i['Datum'].dt.weekday
   i['Tag']=i['Datum'].dt.day
   i['Jahr']=i['Datum'].dt.year
   i['Monat']=i['Datum'].dt.month
   i['Wochenende']=i['Wochentag'].apply(lambda x: 'Wochentag' if x in (0,1,2,3,4) else 'Wochenende')
    

### Adding a Gesamt Column

In [226]:
#Adding a Gesamt column to all the new dataframes which contains the total sales
dataframe=[df_new,df_new_1,df_new_2,df_new_3]

for i in dataframe:
    i['Gesamt']=i['Vollkornbrot']+i['Mischbrot']+i['Stangenbrote']+i['Weizenbrot']+i['Spezialbrot']+i['Brötchen']
    +i['Süsse_Brötchen']+i['Herzhafte_Brötchen']+i['KonditoreiBlech1']+i['KonditoreiBlech2']+i['Blechkuchen']
    +i['Weihnachtsartikel']+i['Stückgebäck']

In [227]:
df_new_index = df_new.set_index('Datum')


### Definition of KonditoreiBlech as Sum

In [228]:
#KonditoreiBlech1 is only sold until 2019, so I build the sum of both caketypes
for i in dataframe:
  i['KonditoreiBlech1']=i['KonditoreiBlech1']+i['KonditoreiBlech2']
  i.rename(columns={'KonditoreiBlech1':'KonditoreiBlech'}, inplace=True)
  i.drop('KonditoreiBlech2',axis='columns', inplace=True)

# Correlations

In [229]:
#First we create a new DataFrame with only numerical features
numerics = ['int64', 'float64']

num_df_2 = df_new_2.select_dtypes(include=numerics)

In [ ]:
#Korrelationsanalyse
mask = np.triu( num_df_2.corr())
plt.figure(figsize=(40,30))
plt.title('Correlation of variables')
ax=sns.heatmap(num_df_2.corr().round(2),vmax=1.0,annot=True,mask=mask,cmap='coolwarm')
plt.savefig('correlogram_simple.png')
ax.set_ylim((0,37))
plt.show

# Feature Engineering

In [230]:
#Change of the Wochenende column
df_new['Wochenende_flag']=df_new['Wochenende'].apply(lambda x: 1 if x=='Wochenende' else 0)

In [231]:
#Change of the Wochenende column
df_new_1['Wochenende_flag']=df_new_1['Wochenende'].apply(lambda x: 1 if x=='Wochenende' else 0)

In [232]:
#Change of the Wochenende column
df_new_2['Wochenende_flag']=df_new_2['Wochenende'].apply(lambda x: 1 if x=='Wochenende' else 0)
#df_new_2['Wochenende_flag']=pd.to_numeric(df_new_2['Wochenende'])

In [233]:
#Change of the Wochenende column
df_new_3['Wochenende_flag']=df_new_3['Wochenende'].apply(lambda x: 1 if x=='Wochenende' else 0)
#df_new_3['Wochenende_flag']=pd.to_numeric(df_new_3['Wochenende'])

In [234]:
df_new.drop(columns=['Wochenende'],inplace=True)

In [235]:
df_new_1.drop(columns=['Wochenende'],inplace=True)

In [236]:
df_new_2.drop(columns=['Wochenende'],inplace=True)

In [237]:
df_new_3.drop(columns=['Wochenende'],inplace=True)

## Season

In [238]:
def season_of_date(date):
    year = str(date.year)
    seasons = {'spring': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'summer': pd.date_range(start='21/06/'+year, end='22/09/'+year),
               'autumn': pd.date_range(start='23/09/'+year, end='20/12/'+year)}
    if date in seasons['spring']:
        return '1'
    if date in seasons['summer']:
        return '2'
    if date in seasons['autumn']:
        return '3'
    else:
        return '4'

In [239]:
# Assuming df has a date column of type `datetime`
df_new['Season'] = df_new.Datum.map(season_of_date)
df_new_1['Season'] = df_new_1.Datum.map(season_of_date)
df_new_2['Season'] = df_new_2.Datum.map(season_of_date)
df_new_3['Season'] = df_new_3.Datum.map(season_of_date)

In [240]:
df_new['Season']=pd.to_numeric(df_new['Season'])
df_new_1['Season']=pd.to_numeric(df_new_1['Season'])
df_new_2['Season']=pd.to_numeric(df_new_2['Season'])
df_new_1['Season']=pd.to_numeric(df_new_3['Season'])

## Chrismas Season Flag

In [241]:
df_new_1['Christmas'] =df_new_1['Monat'].apply(lambda x: 1 if x in (10,11,12,1) else 0)

In [242]:
df_new_2['Christmas'] =df_new_2['Monat'].apply(lambda x: 1 if x in (10,11,12,1) else 0)

In [243]:
df_new_3['Christmas'] =df_new_3['Monat'].apply(lambda x: 1 if x in (10,11,12,1) else 0)

## Closed Flag

In [244]:
df_new['Closed']=df_new['Gesamt'].apply(lambda x: 1 if x==0 else 0)
df_new['Closed']=pd.to_numeric(df_new['Closed'])

In [245]:
df_new_1['Closed']=df_new_1['Gesamt'].apply(lambda x: 1 if x==0 else 0)
df_new_1['Closed']=pd.to_numeric(df_new_1['Closed'])

In [246]:
df_new_2['Closed']=df_new_2['Gesamt'].apply(lambda x: 1 if x==0 else 0)
df_new_2['Closed']=pd.to_numeric(df_new_2['Closed'])

In [247]:
df_new_3['Closed']=df_new_3['Gesamt'].apply(lambda x: 1 if x==0 else 0)
df_new_3['Closed']=pd.to_numeric(df_new_3['Closed'])

### Closed for Prophet

In [ ]:
CLOSED=df_new_2.query('Gesamt==0').Datum

In [ ]:
closed=[i.strftime(format = "%Y-%m-%d") for i in CLOSED]

In [ ]:
Closed = pd.DataFrame({
  'holiday': 'Closed',
  'ds': pd.to_datetime(['2015-01-01',
 '2015-04-03',
 '2015-05-01',
 '2015-12-25',
 '2015-12-26',
 '2016-01-01',
 '2016-03-24',
 '2016-03-25',
 '2016-07-07',
 '2016-07-18',
 '2016-08-14',
 '2016-08-15',
 '2016-12-17',
 '2016-12-25',
 '2016-12-26',
 '2017-01-01',
 '2017-04-14',
 '2017-05-01',
 '2017-07-04',
 '2017-10-23',
 '2017-10-31',
 '2017-12-25',
 '2017-12-26',
 '2018-01-01',
 '2018-03-30',
 '2018-05-01',
 '2018-05-21',
 '2018-10-31',
 '2018-12-25',
 '2018-12-26',
 '2019-01-01',
 '2019-03-20',
 '2019-04-03',
 '2019-04-19',
 '2019-05-01',
 '2019-07-10',
 '2019-10-31',
 '2019-12-25',
 '2019-12-26',
 '2020-01-01',
 '2020-04-10',
 '2020-05-01',
 '2020-05-03',
 '2020-05-06',
 '2020-05-07']),
  'lower_window': 0,
  'upper_window': 1,
})

In [ ]:
Closed

In [ ]:
HOLIDAYS_SH_SOMMER

## Holidays

### Dates of special interest

In [248]:
def def_out(Monat,Tag):
    ergebnis=0
    if Monat==12 and Tag in (30,31,24):
        ergebnis=1
    else:
        ergebnis=0
    return ergebnis    

In [249]:
liste=[df_new_1,df_new_2,df_new_3,df_new]

for i in liste:
   
   i['Outlier']=i.apply(lambda row: def_out(row['Monat'],row['Tag']), axis=1)

In [250]:
df_new.Outlier.value_counts()

0    5487
1      42
Name: Outlier, dtype: int64

### New Years Eve

In [251]:
def def_nye(Monat,Tag):
    ergebnis=0
    if Monat==12 and Tag==31:
        ergebnis=1
    else:
        ergebnis=0
    return ergebnis   

In [252]:
liste=[df_new_1,df_new_2,df_new_3,df_new]

for i in liste:
   
   i['New_Years_Eve']=i.apply(lambda row: def_nye(row['Monat'],row['Tag']), axis=1)

### Public Celebrationdays via package holidays

In [253]:
#Definition of Feiertage
de_celebration_days=[]
for date in holidays.Germany(years=2020).items():
    de_celebration_days.append(str(date[0]))
for date in holidays.Germany(years =2019).items():
    de_celebration_days.append(str(date[0]))
for date in holidays.Germany(years=2018).items():
    de_celebration_days.append(str(date[0]))
for date in holidays.Germany(years=2017).items():
    de_celebration_days.append(str(date[0]))
for date in holidays.Germany(years=2016).items():
    de_celebration_days.append(str(date[0]))
for date in holidays.Germany(years=2015).items():
    de_celebration_days.append(str(date[0]))     

In [254]:
#Definition of new column Feiertage_DE
df_new['Feiertag_DE']=df_new['Datum'].apply(lambda x: 1 if x.strftime(format = "%Y-%m-%d") in de_celebration_days else 0)

In [255]:
#Definition of new column Feiertage_DE
df_new_1['Feiertag_DE']=df_new_1['Datum'].apply(lambda x: 1 if x.strftime(format = "%Y-%m-%d") in de_celebration_days else 0)

In [256]:
#Definition of new column Feiertage_DE
df_new_2['Feiertag_DE']=df_new_2['Datum'].apply(lambda x: 1 if x.strftime(format = "%Y-%m-%d") in de_celebration_days else 0)

In [257]:
#Definition of new column Feiertage_DE
df_new_3['Feiertag_DE']=df_new_3['Datum'].apply(lambda x: 1 if x.strftime(format = "%Y-%m-%d") in de_celebration_days else 0)

### Ostern

In [200]:
de_celebration_days

['2020-01-01',
 '2020-04-10',
 '2020-04-13',
 '2020-05-01',
 '2020-05-21',
 '2020-06-01',
 '2020-10-03',
 '2020-12-25',
 '2020-12-26',
 '2019-01-01',
 '2019-04-19',
 '2019-04-22',
 '2019-05-01',
 '2019-05-30',
 '2019-06-10',
 '2019-10-03',
 '2019-12-25',
 '2019-12-26',
 '2018-01-01',
 '2018-03-30',
 '2018-04-02',
 '2018-05-01',
 '2018-05-10',
 '2018-05-21',
 '2018-10-03',
 '2018-12-25',
 '2018-12-26',
 '2017-01-01',
 '2017-04-14',
 '2017-04-17',
 '2017-05-01',
 '2017-05-25',
 '2017-06-05',
 '2017-10-03',
 '2017-10-31',
 '2017-12-25',
 '2017-12-26',
 '2016-01-01',
 '2016-03-25',
 '2016-03-28',
 '2016-05-01',
 '2016-05-05',
 '2016-05-16',
 '2016-10-03',
 '2016-12-25',
 '2016-12-26',
 '2015-01-01',
 '2015-04-03',
 '2015-04-06',
 '2015-05-01',
 '2015-05-14',
 '2015-05-25',
 '2015-10-03',
 '2015-12-25',
 '2015-12-26']

### Before Ostern

In [258]:
de_ostern = pd.DataFrame({'date':pd.date_range('2019-04-20', '2019-04-20')})
de_ostern = de_ostern.append(pd.DataFrame({'date':pd.date_range('2019-04-18', '2018-04-18')})) 
de_ostern = de_ostern.append(pd.DataFrame({'date':pd.date_range('2018-03-31', '2018-03-31')}))
de_ostern = de_ostern.append(pd.DataFrame({'date':pd.date_range('2018-03-29', '2018-03-29')}))
de_ostern = de_ostern.append(pd.DataFrame({'date':pd.date_range('2017-04-15', '2017-04-15')}))
de_ostern = de_ostern.append(pd.DataFrame({'date':pd.date_range('2017-04-13', '2017-04-13')}))
de_ostern = de_ostern.append(pd.DataFrame({'date':pd.date_range('2016-03-30', '2016-03-30')}))
de_ostern = de_ostern.append(pd.DataFrame({'date':pd.date_range('2016-03-28', '2016-03-28')}))
de_ostern = de_ostern.append(pd.DataFrame({'date':pd.date_range('2015-04-04', '2015-04-04')}))
de_ostern = de_ostern.append(pd.DataFrame({'date':pd.date_range('2015-04-02', '2015-04-02')}))
de_ostern = de_ostern.sort_values('date') 
de_ostern['Ostern']=1

In [259]:
df_new_1=pd.merge(df_new_1,de_ostern,left_on='Datum',right_on='date',how='left').fillna(0)

In [260]:
df_new_2=pd.merge(df_new_2,de_ostern,left_on='Datum',right_on='date',how='left').fillna(0)

In [261]:
df_new_3=pd.merge(df_new_3,de_ostern,left_on='Datum',right_on='date',how='left').fillna(0)

In [262]:
df_new_1.drop(columns=['date'],inplace=True)
df_new_2.drop(columns=['date'],inplace=True)
df_new_3.drop(columns=['date'],inplace=True)

### School holidays SH

In [263]:
summer_sh = pd.DataFrame({'date':pd.date_range('2019-12-23', '2020-01-06')})                  
summer_sh = summer_sh.append(pd.DataFrame({'date':pd.date_range('2019-07-01', '2019-08-08')}))                                   
summer_sh = summer_sh.append(pd.DataFrame({'date':pd.date_range('2018-07-09', '2018-08-17')}))               
summer_sh = summer_sh.append(pd.DataFrame({'date':pd.date_range('2017-07-24', '2017-09-02')}))               
summer_sh = summer_sh.append(pd.DataFrame({'date':pd.date_range('2016-07-25', '2016-09-02')}))               
summer_sh = summer_sh.append(pd.DataFrame({'date':pd.date_range('2015-07-20', '2015-08-29')}))                
summer_sh = summer_sh.sort_values('date') 
summer_sh['SUMMER_SH']=1


In [264]:
holidays_sh = pd.DataFrame({'date':pd.date_range('2019-12-23', '2020-01-06')})
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2020-03-31', '2020-04-17')}))
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2018-12-10', '2019-01-04')}))
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2019-04-04', '2019-04-18')}))
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2019-05-30', '2019-05-30')}))                    
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2019-07-01', '2019-08-08')}))               
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2019-10-05', '2019-10-17')}))
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2017-12-21', '2018-01-06')}))
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2018-03-29', '2018-04-13')}))
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2018-05-11', '2018-05-11')}))                    
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2018-07-09', '2018-08-17')}))               
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2018-10-01', '2018-10-19')}))    
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2016-12-23', '2017-01-06')}))
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2017-04-07', '2017-04-21')}))
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2017-05-26', '2017-05-26')}))                    
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2017-07-24', '2017-09-02')}))               
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2017-10-16', '2017-10-27')}))       
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2015-12-21', '2016-01-06')}))
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2016-03-24', '2016-04-09')}))
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2016-05-16', '2016-05-16')}))                    
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2016-07-25', '2016-09-02')}))               
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2016-10-17', '2016-10-29')}))    
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2014-12-22', '2015-01-06')}))
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2015-04-01', '2015-04-17')}))
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2015-05-11', '2015-05-11')}))                    
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2015-07-20', '2015-08-29')}))               
holidays_sh = holidays_sh.append(pd.DataFrame({'date':pd.date_range('2015-10-19', '2015-10-31')}))    


holidays_sh = holidays_sh.sort_values('date') 
holidays_sh['Holidays_SH']=1


In [265]:
df_new=pd.merge(df_new,holidays_sh,left_on='Datum',right_on='date',how='left').fillna(0)

In [266]:
df_new_1=pd.merge(df_new_1,holidays_sh,left_on='Datum',right_on='date',how='left').fillna(0)
df_new_1=pd.merge(df_new_1,summer_sh,left_on='Datum',right_on='date',how='left').fillna(0)

In [267]:
df_new_2=pd.merge(df_new_2,holidays_sh,left_on='Datum',right_on='date',how='left').fillna(0)
df_new_2=pd.merge(df_new_2,summer_sh,left_on='Datum',right_on='date',how='left').fillna(0)

In [268]:
df_new_3=pd.merge(df_new_3,holidays_sh,left_on='Datum',right_on='date',how='left').fillna(0)
df_new_3=pd.merge(df_new_3,summer_sh,left_on='Datum',right_on='date',how='left').fillna(0)


In [269]:
df_new.drop(columns=['date'],inplace=True)

In [270]:
df_new_1.drop(columns=['date'],inplace=True)

KeyError: "['date'] not found in axis"

In [ ]:
df_new_2.drop(columns=['date'],inplace=True)

In [ ]:
df_new_3.drop(columns=['date'],inplace=True)

#### School Holidays for Prophet

In [ ]:
HOLIDAYS_SH_SOMMER = pd.DataFrame({
  'holiday': 'Sommer',
  'ds': pd.to_datetime(['2015-07-20','2016-07-25','2017-07-24','2018-07-09','2019-07-01']),
  'lower_window': -2,
  'upper_window': 42,
})

In [ ]:
OSTERN = pd.DataFrame({
  'holiday': 'Ostern',
  'ds': pd.to_datetime(['2019-04-04','2018-03-29','2016-03-24','2017-04-07','2015-04-01']),
  'lower_window': 0,
  'upper_window': 18,
})

OSTERN.loc[OSTERN.ds=='2017-04-07','upper_window']=17
OSTERN.loc[OSTERN.ds=='2015-04-01','upper_window']=19
OSTERN.loc[OSTERN.ds=='2019-04-04','upper_window']=15

In [ ]:
OSTERN.loc[OSTERN.ds=='2017-04-07','upper_window']=17
OSTERN.loc[OSTERN.ds=='2015-04-01','upper_window']=19
OSTERN.loc[OSTERN.ds=='2019-04-04','upper_window']=15

In [ ]:
HERBST = pd.DataFrame({
  'holiday': 'Herbst',
  'ds': pd.to_datetime(['2015-10-19','2016-10-17','2017-10-16','2019-10-07','2018-10-01']),
  'lower_window': -2,
  'upper_window': 13,
})
HERBST.loc[HERBST.ds=='2018-10-01','upper_window']=20

In [ ]:
HOLIDAYS_SH_ALL=pd.concat((HERBST, OSTERN))

In [ ]:
holidays_2019 = {'Weihnachten': pd.date_range(start='20/12/2018', end='04/01/2019'),
                     'Ostern': pd.date_range(start='04/04/2019', end='18/04/2019'),
               
               'Herbst': pd.date_range(start='05/10/2019', end='17/10/2019')}
    holidays_2018 = {'Weihnachten': pd.date_range(start='21/12/2017', end='06/01/2018'),
               'Ostern': pd.date_range(start='29/03/2018', end='13/04/2018'),
               
               'Herbst': pd.date_range(start='01/10/2018', end='19/10/2018')}
    holidays_2017 = {'Weihnachten': pd.date_range(start='23/12/2016', end='06/01/2017'),
               'Ostern': pd.date_range(start='07/04/2017', end='21/04/2017'),
               
              
               'Herbst': pd.date_range(start='16/10/2017', end='27/10/2017')}
    holidays_2016 = {'Weihnachten': pd.date_range(start='21/12/2015', end='06/01/2016'),
               'Ostern': pd.date_range(start='24/03/2016', end='09/04/2016'),
               
               'Sommer': pd.date_range(start='25/07/2016', end='17/08/2016'),
               'Herbst': pd.date_range(start='17/10/2016', end='29/10/2016')}
    holidays_2015 = {'Weihnachten': pd.date_range(start='22/12/2014', end='06/01/2015'),
               'Ostern': pd.date_range(start='01/04/2015', end='17/04/2015'),
               
                     
               'Herbst': pd.date_range(start='19/10/2015', end='31/10/2015')}
    if date in holidays_2020['Weihnachten']:

#### Kieler Woche for Prophet

In [39]:
Kieler_Woche= pd.DataFrame({
  'holiday': 'Kieler_Woche',
  'ds': pd.to_datetime(['2015-06-20','2016-06-18','2017-06-17','2018-06-16','2019-06-22']),
  'lower_window': 0,
  'upper_window': 8,
})

In [40]:
Kieler_Woche

,holiday,ds,lower_window,upper_window
0,Kieler_Woche,2015-06-20,0,8
1,Kieler_Woche,2016-06-18,0,8
2,Kieler_Woche,2017-06-17,0,8
3,Kieler_Woche,2018-06-16,0,8
4,Kieler_Woche,2019-06-22,0,8


### Kieler Woche

In [ ]:
kieler_woche= pd.DataFrame({'date':pd.date_range('2019-06-22', '2019-06-30')})
kieler_woche = kieler_woche.append(pd.DataFrame({'date':pd.date_range('2018-06-16', '2018-06-24')}))
kieler_woche = kieler_woche.append(pd.DataFrame({'date':pd.date_range('2017-06-17', '2017-06-25')}))
kieler_woche = kieler_woche.append(pd.DataFrame({'date':pd.date_range('2016-06-18', '2016-06-26')}))
kieler_woche = kieler_woche.append(pd.DataFrame({'date':pd.date_range('2015-06-20', '2015-06-28')}))

In [ ]:
kieler_woche['Datum']=kieler_woche['date'].apply(lambda x: x.strftime(format = "%Y-%m-%d"))
liste_kw=kieler_woche['Datum'].tolist()

In [ ]:
#Definition of new column Kieler Woche
df_new['Kieler_Woche']=df_new['Datum'].apply(lambda x: 1 if x.strftime(format = "%Y-%m-%d") in liste_kw else 0)

In [ ]:
#Definition of new column Kieler Woche
df_new_1['Kieler_Woche']=df_new_1['Datum'].apply(lambda x: 1 if x.strftime(format = "%Y-%m-%d") in liste_kw else 0)

In [ ]:
#Definition of new column Kieler Woche
df_new_2['Kieler_Woche']=df_new_2['Datum'].apply(lambda x: 1 if x.strftime(format = "%Y-%m-%d") in liste_kw else 0)

In [ ]:
##Definition of new column Kieler Woche
df_new_3['Kieler_Woche']=df_new_3['Datum'].apply(lambda x: 1 if x.strftime(format = "%Y-%m-%d") in liste_kw else 0)

In [ ]:
df_new_2['Kieler_Woche'].value_counts()

### Further Use

In [47]:
import holidays

#prov = BW, BY, BE, BB, HB, HH, HE, MV, NI, NW, RP, SL, SN, ST, SH, TH

holidays_SH = holidays.CountryHoliday('DE', prov='SH', state=None )


## Verkaufsfreie Sonntage

## Wheather data from dwd/weste

In [310]:
df_wetter = pd.read_csv("Wetterdaten_Kiel_gesamt.csv",delimiter=';')

In [311]:
df_wetter.head(5)

,Datum,Niederschlag,Sonne_h,Temperatur_max,Unnamed: 4
0,01.01.2015,8,0,"6,2",NaN
1,02.01.2015,"2,2","0,7","9,5",NaN
2,03.01.2015,"4,2",0,"5,9",NaN
3,04.01.2015,0,"4,7","5,8",NaN
4,05.01.2015,"0,2",0,"6,1",NaN


In [312]:
df_wetter.drop(columns=['Unnamed: 4'],inplace=True)

In [313]:
df_wetter.Datum=pd.to_datetime(df_wetter['Datum'])
df_wetter['Datum']=df_wetter['Datum'].apply(lambda x: x.strftime(format = "%Y-%m-%d") )
df_wetter.Datum=pd.to_datetime(df_wetter['Datum'])

In [314]:
df_wetter.rename(columns={'Datum':'date'}, inplace=True)

In [315]:
df_wetter.Niederschlag=df_wetter.Niederschlag.apply(lambda x: str(x).replace(",", "."))
df_wetter.Sonne_h=df_wetter.Sonne_h.apply(lambda x: str(x).replace(",", "."))
df_wetter.Sonne_h=df_wetter.Sonne_h.apply(lambda x: str(x).replace("nan", "0"))
df_wetter.Temperatur_max=df_wetter.Temperatur_max.apply(lambda x: str(x).replace(",", "."))

In [316]:
df_new=pd.concat([df_new,df_wetter],axis=1).fillna(0)

In [317]:
df_new_1=pd.concat([df_new_1,df_wetter],axis=1).fillna(0)

In [318]:
df_new_2=pd.concat([df_new_2,df_wetter],axis=1).fillna(0)

In [319]:
df_new_3=pd.concat([df_new_3,df_wetter],axis=1).fillna(0)

In [320]:
df_new['Niederschlag']=pd.to_numeric(df_new['Niederschlag'])
df_new['Sonne_h']=pd.to_numeric(df_new['Sonne_h'])
df_new['Temperatur_max']=pd.to_numeric(df_new['Temperatur_max'])

In [321]:
df_new_1['Niederschlag']=pd.to_numeric(df_new_1['Niederschlag'])
df_new_1['Sonne_h']=pd.to_numeric(df_new_1['Sonne_h'])
df_new_1['Temperatur_max']=pd.to_numeric(df_new_1['Temperatur_max'])

In [322]:
df_new_2['Niederschlag']=pd.to_numeric(df_new_2['Niederschlag'])
df_new_2['Sonne_h']=pd.to_numeric(df_new_2['Sonne_h'])
df_new_2['Temperatur_max']=pd.to_numeric(df_new_2['Temperatur_max'])

In [323]:
df_new_3['Niederschlag']=pd.to_numeric(df_new_3['Niederschlag'])
df_new_3['Sonne_h']=pd.to_numeric(df_new_3['Sonne_h'])
df_new_3['Temperatur_max']=pd.to_numeric(df_new_3['Temperatur_max'])

In [324]:
df_new.head(2)

,Datum,Filiale,Weizenbrot,Mischbrot,Vollkornbrot,Stangenbrote,Spezialbrot,Brötchen,Süsse_Brötchen,Herzhafte_Brötchen,...,Season,Closed,Outlier,New_Years_Eve,Feiertag_DE,Holidays_SH,date,Niederschlag,Sonne_h,Temperatur_max
0,2015-01-02,1,39.994964,152.610944,121.284522,118.786757,0.000000,220.142630,181.329660,163.713061,...,4,0,0,0,0,1.0,2015-01-01 00:00:00,8.0,0.0,6.2
1,2015-01-02,2,88.023750,84.633190,115.157621,58.999279,66.687652,505.565569,235.597443,87.759013,...,4,0,0,0,0,1.0,2015-02-01 00:00:00,2.2,0.7,9.5


In [325]:
df_new.drop(columns=['date'],inplace=True)

In [326]:
df_new_1.drop(columns=['date'],inplace=True)

In [327]:
df_new_2.drop(columns=['date'],inplace=True)

In [328]:
df_new_3.drop(columns=['date'],inplace=True)

## Timerelated Values

### Values of the previous Week and Jahr

In [ ]:
#Value of the previous Year
df_new_2['Brötchen_364'] = df_new_2.Brötchen.shift(364)
df_new_2['Brötchen_364'].fillna(0)

In [ ]:
#Value of the previous Week
df_new_2['Brötchen_7'] = df_new_2.Brötchen.shift(7)
df_new_2['Brötchen_7'].fillna(0)

In [ ]:
#For multiple Columns and also Dataframes
var=['Süsse_Brötchen','Herzhafte_Brötchen']
numbers=[7,364]

for i in var:
    for j in numbers:
       df_new_2["%s_%d" % (i,j)]=df_new_2[i].shift(j)
       df_new_2["%s_%d" % (i,j)].fillna(0)

### mean and max-Values for Weekday and Monthly Values

**Conclusions**
- `Mean and Maxvalues have no significant impact to the correlation. I need to do it for Months and Weekdays.`


In [ ]:
def Brötchen_mean(x):
    return df_new_2.groupby('Wochentag')['Brötchen'].mean()[x]
def Brötchen_max(x):
    return df_new_2.groupby('Wochentag')['Brötchen'].max()[x]


In [ ]:
df_new_2['Brötchen_mean_WD']=df_new_2['Wochentag'].apply(lambda x:  Brötchen_mean(x))
df_new_2['Brötchen_max_WD']=df_new_2['Wochentag'].apply(lambda x:  Brötchen_max(x))


In [ ]:
#For multiple Columns and also Dataframes
var=['Süsse_Brötchen','Herzhafte_Brötchen']


for i in var:
    name=i+ "_mean"
    
    def name(x):
        return df_new_2.groupby('Wochentag')['%s' % (i)].mean()[x]

    df_new_2['%s_mean_WD' % (i)]=df_new_2['Wochentag'].apply(lambda x:  name(x))
                                                          

In [ ]:
def Brötchen_mean_M(x):
    return df_new_2.groupby('Monat')['Brötchen'].mean()[x]
def Brötchen_max_M(x):
    return df_new_2.groupby('Monat')['Brötchen'].max()[x]


In [ ]:
df_new_2['Brötchen_mean_M']=df_new_2['Monat'].apply(lambda x:  Brötchen_mean_M(x))
df_new_2['Brötchen_max_M']=df_new_2['Monat'].apply(lambda x:  Brötchen_max_M(x))


In [331]:
df_new_1.drop(columns=['date_x','date_y'],inplace=True)

In [330]:
df_new_2.drop(columns=['date_x','date_y'],inplace=True)

In [332]:
df_new_3.drop(columns=['date_x','date_y'],inplace=True)

# Outlier Handling

## Filiale 2

In [333]:
data_2_clean=df_new_2.copy()

In [334]:
data_2_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1979 entries, 0 to 1978
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Datum               1979 non-null   datetime64[ns]
 1   Weizenbrot          1979 non-null   float64       
 2   Mischbrot           1979 non-null   float64       
 3   Vollkornbrot        1979 non-null   float64       
 4   Stangenbrote        1979 non-null   float64       
 5   Spezialbrot         1979 non-null   float64       
 6   Brötchen            1979 non-null   float64       
 7   Süsse_Brötchen      1979 non-null   float64       
 8   Herzhafte_Brötchen  1979 non-null   float64       
 9   KonditoreiBlech     1979 non-null   float64       
 10  Stückgebäck         1979 non-null   float64       
 11  Blechkuchen         1979 non-null   float64       
 12  Weihnachtsartikel   1979 non-null   float64       
 13  Wochentag           1979 non-null   int64       

### Weizenbrot

In [ ]:
Weizenbrot_result = np.percentile(df_new_2.Weizenbrot, q=[0, 25, 50, 75, 90, 95,99])
print(Weizenbrot_result)

In [ ]:
df_new_2.query('Weizenbrot>106.46010925')[['Datum','Weizenbrot','Wochenende_flag','Feiertag_DE','Holidays_SH']]

In [335]:
data_2_clean['Weizenbrot']=data_2_clean['Weizenbrot'].apply(lambda x: x if x <106.46010925 else None )

### Mischbrot

In [336]:
data_2_clean['Mischbrot']=data_2_clean['Mischbrot'].apply(lambda x: x if x >0 else 0 )

In [273]:
df_new_2.Mischbrot.describe(percentiles=[ 0.9, 0.99])

count    1979.000000
mean       71.860719
std        20.052765
min         0.000000
50%        73.463033
90%        94.252644
99%       116.017187
max       152.746146
Name: Mischbrot, dtype: float64

In [274]:
#df_new_2.query('Mischbrot>116')[['Datum','Mischbrot','Wochenende_flag','Feiertag_DE','Holidays_SH']]

In [337]:
data_2_clean['Mischbrot']=data_2_clean['Mischbrot'].apply(lambda x: x if x <116 else None )

### Vollkornbrot

In [276]:
df_new_2.Vollkornbrot.describe(percentiles=[0.9, 0.99])

count    1979.000000
mean      108.283982
std        26.926271
min         0.000000
50%       108.428509
90%       139.188596
99%       165.980698
max       225.035588
Name: Vollkornbrot, dtype: float64

In [277]:
#df_new_2.query('Vollkornbrot>165.980698')[['Datum','Vollkornbrot','Wochenende_flag','Feiertag_DE','Holidays_SH']]

In [338]:
data_2_clean['Vollkornbrot']=data_2_clean['Vollkornbrot'].apply(lambda x: x if x <165.980698 else None )

### Spezialbrot

In [279]:
df_new_2.Spezialbrot.describe(percentiles=[0.9, 0.99])

count    1979.000000
mean       56.310284
std        26.874605
min         0.000000
50%        61.850288
90%        80.372255
99%       106.508961
max       200.259200
Name: Spezialbrot, dtype: float64

In [280]:
#df_new_2.query('Spezialbrot>106.508961')[['Datum','Spezialbrot','Wochenende_flag','Feiertag_DE','Holidays_SH']]

In [339]:
data_2_clean['Spezialbrot']=data_2_clean['Spezialbrot'].apply(lambda x: x if x <165.980698 else None )

### Stangenbrote

In [282]:
df_new_2.Stangenbrote.describe(percentiles=[0.9, 0.99])

count    1979.000000
mean       39.816439
std        17.646559
min         0.000000
50%        39.332197
90%        59.523275
99%        82.606569
max       198.752191
Name: Stangenbrote, dtype: float64

In [283]:
#df_new_2.query('Stangenbrote>82.606569')[['Datum','Stangenbrote','Wochenende_flag','Feiertag_DE','Holidays_SH']]

In [340]:
data_2_clean['Stangenbrote']=data_2_clean['Stangenbrote'].apply(lambda x: x if x <82.606569 else None )

### Brötchen

In [285]:
df_new_2.Brötchen.describe(percentiles=[0.9, 0.99])

count    1979.000000
mean      375.431206
std       138.927415
min         0.000000
50%       355.500769
90%       573.698524
99%       736.781705
max       820.516093
Name: Brötchen, dtype: float64

In [286]:
df_new_2.query('Brötchen>736.781705')[['Datum','Brötchen','Wochenende_flag','Feiertag_DE','Holidays_SH']]

,Datum,Brötchen,Wochenende_flag,Feiertag_DE,Holidays_SH
95,2015-04-06,737.004266,0,1,1.0
177,2015-06-27,743.175388,1,0,0.0
206,2015-07-26,739.086126,1,0,1.0
213,2015-08-02,770.762031,1,0,1.0
541,2016-06-25,772.567025,1,0,0.0
1297,2018-07-21,769.241046,1,0,1.0
1325,2018-08-18,766.553081,1,0,0.0
1570,2019-04-20,810.818652,1,0,0.0
1571,2019-04-21,762.131477,1,0,0.0
1572,2019-04-22,744.701161,0,1,0.0


In [341]:
data_2_clean['Brötchen']=data_2_clean['Brötchen'].apply(lambda x: x if x <736.781705 else None )

### Süsse Brötchen

In [288]:
df_new_2.Süsse_Brötchen.describe(percentiles=[0.9, 0.99])

count    1979.000000
mean      196.670399
std        57.579214
min         0.000000
50%       185.408152
90%       274.316402
99%       345.075359
max       428.071519
Name: Süsse_Brötchen, dtype: float64

In [289]:
df_new_2.query('Süsse_Brötchen>345.075359')[['Datum','Süsse_Brötchen','Wochenende_flag','Feiertag_DE','Holidays_SH']]

,Datum,Süsse_Brötchen,Wochenende_flag,Feiertag_DE,Holidays_SH
233,2015-08-22,348.657567,1,0,1.0
576,2016-07-30,360.564346,1,0,1.0
590,2016-08-13,376.487824,1,0,1.0
597,2016-08-20,428.071519,1,0,1.0
940,2017-07-29,378.896359,1,0,1.0
947,2017-08-05,350.236377,1,0,1.0
1298,2018-07-22,348.069307,1,0,1.0
1311,2018-08-04,348.281888,1,0,1.0
1312,2018-08-05,362.384869,1,0,1.0
1319,2018-08-12,346.946151,1,0,1.0


In [342]:
data_2_clean['Süsse_Brötchen']=data_2_clean['Süsse_Brötchen'].apply(lambda x: x if x <345.075359 else None )

### Herzhafte Brötchen

In [291]:
df_new_2.Herzhafte_Brötchen.describe(percentiles=[0.9, 0.99])

count    1979.000000
mean       93.947422
std        36.940020
min         0.000000
50%        87.759013
90%       142.352808
99%       203.997358
max       268.706655
Name: Herzhafte_Brötchen, dtype: float64

In [292]:
df_new_2.query('Herzhafte_Brötchen>203.997358')[['Datum','Herzhafte_Brötchen','Wochenende_flag','Feiertag_DE','Holidays_SH']]

,Datum,Herzhafte_Brötchen,Wochenende_flag,Feiertag_DE,Holidays_SH
1291,2018-07-15,205.004841,1,0,1.0
1298,2018-07-22,218.275924,1,0,1.0
1306,2018-07-30,208.208486,0,0,1.0
1311,2018-08-04,232.411259,1,0,1.0
1325,2018-08-18,218.438997,1,0,0.0
1326,2018-08-19,209.905315,1,0,0.0
1543,2019-03-24,213.660279,1,0,0.0
1633,2019-06-22,205.960300,1,0,0.0
1634,2019-06-23,221.737360,1,0,0.0
1640,2019-06-29,244.708756,1,0,0.0


In [343]:
data_2_clean['Herzhafte_Brötchen']=data_2_clean['Herzhafte_Brötchen'].apply(lambda x: x if x <203.997358 else 
                                                                             None )

### KonditoreiBlech

In [294]:
df_new_2.KonditoreiBlech.describe(percentiles=[0.9, 0.99])

count    1979.000000
mean       98.692646
std        64.111577
min         0.000000
50%        73.102484
90%       193.462493
99%       275.632008
max       399.988543
Name: KonditoreiBlech, dtype: float64

In [295]:
df_new_2.query('KonditoreiBlech>275.632008')[['Datum','KonditoreiBlech','Wochenende_flag','Feiertag_DE','Holidays_SH']]

,Datum,KonditoreiBlech,Wochenende_flag,Feiertag_DE,Holidays_SH
17,2015-01-18,326.082164,1,0,0.0
31,2015-02-01,284.671715,1,0,0.0
38,2015-02-08,351.096954,1,0,0.0
52,2015-02-22,399.988543,1,0,0.0
66,2015-03-08,296.794889,1,0,0.0
332,2015-11-29,307.686998,1,0,0.0
399,2016-02-04,288.379119,0,0,0.0
402,2016-02-07,278.672843,1,0,0.0
408,2016-02-13,283.452860,1,0,0.0
409,2016-02-14,347.929304,1,0,0.0


In [344]:
data_2_clean['KonditoreiBlech']=data_2_clean['KonditoreiBlech'].apply(lambda x: x if x <275.632008 else None )

### Blechkuchen

In [297]:
df_new_2.Blechkuchen.describe(percentiles=[0.9, 0.99])

count    1979.000000
mean       93.154924
std        46.648699
min         0.000000
50%        87.470191
90%       155.746063
99%       222.225412
max       338.496148
Name: Blechkuchen, dtype: float64

In [298]:
df_new_2.query('Blechkuchen>222.225412')[['Datum','Blechkuchen','Wochenende_flag','Feiertag_DE','Holidays_SH']]

,Datum,Blechkuchen,Wochenende_flag,Feiertag_DE,Holidays_SH
144,2015-05-25,237.669884,0,1,0.0
157,2015-06-07,227.839234,1,0,0.0
247,2015-09-05,275.758250,1,0,0.0
261,2015-09-19,250.168163,1,0,0.0
590,2016-08-13,284.154770,1,0,1.0
597,2016-08-20,247.344333,1,0,1.0
954,2017-08-12,338.496148,1,0,1.0
955,2017-08-13,282.968118,1,0,1.0
962,2017-08-20,263.313403,1,0,1.0
1269,2018-06-23,293.819720,1,0,0.0


In [345]:
data_2_clean['Blechkuchen']=data_2_clean['Blechkuchen'].apply(lambda x: x if x <222.225412 else None)

### Stückgebäck

In [300]:
df_new_2.Stückgebäck.describe(percentiles=[0.9, 0.99])

count    1979.000000
mean      152.085738
std        89.090462
min         0.000000
50%       145.715536
90%       203.852504
99%       303.260565
max      1798.588913
Name: Stückgebäck, dtype: float64

In [301]:
data_2_clean.columns

Index(['Datum', 'Weizenbrot', 'Mischbrot', 'Vollkornbrot', 'Stangenbrote',
       'Spezialbrot', 'Brötchen', 'Süsse_Brötchen', 'Herzhafte_Brötchen',
       'KonditoreiBlech', 'Stückgebäck', 'Blechkuchen', 'Weihnachtsartikel',
       'Wochentag', 'Tag', 'Jahr', 'Monat', 'Gesamt', 'Wochenende_flag',
       'Season', 'Closed', 'Outlier', 'Feiertag_DE', 'Holidays_SH',
       'Kieler_Woche', 'Niederschlag', 'Sonne_h', 'Temperatur_max',
       'Brötchen>', 'Süsse_Brötchen>', 'Herzhafte_Brötchen>',
       'KonditoreiBlech>', 'Blechkuchen>'],
      dtype='object')

In [302]:
data_2_clean.query('Stückgebäck>303.260565')
#[['Datum','Stückgebäck','Wochenende_flag','Feiertag_DE','Holidays_SH','Outlier']]

,Datum,Weizenbrot,Mischbrot,Vollkornbrot,Stangenbrote,Spezialbrot,Brötchen,Süsse_Brötchen,Herzhafte_Brötchen,KonditoreiBlech,...,Holidays_SH,Kieler_Woche,Niederschlag,Sonne_h,Temperatur_max,Brötchen>,Süsse_Brötchen>,Herzhafte_Brötchen>,KonditoreiBlech>,Blechkuchen>
363,2015-12-30,74.207039,84.776793,110.678033,44.484650,93.489671,513.542454,222.236631,95.773345,70.119212,...,1.0,0,0.0,0.000,6.0,513.542454,222.236631,95.773345,70.119212,0.000000
364,2015-12-31,80.696523,103.007808,111.867599,29.439600,150.641623,621.392020,223.927194,104.438491,0.000000,...,1.0,0,1.8,0.000,5.9,621.392020,223.927194,104.438491,0.000000,0.000000
729,2016-12-30,75.684263,89.247237,116.589440,33.456708,67.554767,482.435003,195.119217,37.811926,134.197804,...,1.0,0,0.0,2.800,2.9,482.435003,195.119217,37.811926,134.197804,0.000000
730,2016-12-31,NaN,79.836849,115.958559,42.671246,161.978059,573.556869,237.232767,68.495242,122.198305,...,1.0,0,0.1,0.000,5.7,573.556869,237.232767,68.495242,122.198305,0.000000
1094,2017-12-30,74.016200,91.435729,138.645026,36.205173,109.960417,488.780540,205.471655,94.864324,85.151008,...,1.0,0,2.4,0.000,5.6,488.780540,205.471655,94.864324,85.151008,29.090461
1095,2017-12-31,85.929438,49.851079,95.192115,0.000000,NaN,583.322137,224.616688,80.509262,40.254108,...,1.0,0,14.5,0.000,11.5,583.322137,224.616688,80.509262,40.254108,47.570151
1459,2018-12-30,0.000000,41.313901,102.979151,12.713490,62.880604,510.098921,242.113512,124.080903,63.215898,...,1.0,0,0.0,3.783,8.2,510.098921,242.113512,124.080903,63.215898,40.548101
1460,2018-12-31,77.031724,71.343652,130.801084,37.688275,NaN,619.135110,252.557643,121.648554,0.000000,...,1.0,0,0.3,0.000,8.2,619.135110,252.557643,121.648554,0.000000,0.000000
1824,2019-12-30,91.048103,86.728311,153.868426,8.227841,81.994086,577.170659,206.088817,184.756249,79.538165,...,1.0,0,0.1,0.000,7.8,577.170659,206.088817,184.756249,79.538165,0.000000
1825,2019-12-31,84.140637,79.728814,120.497757,32.527453,161.209522,684.935898,257.226027,107.078955,0.000000,...,1.0,0,0.0,0.000,7.9,684.935898,257.226027,107.078955,0.000000,0.000000


In [303]:
def def_sales(sales,out):
    ergebnis=0
    if out==1 or sales<303.260565:
        ergebnis=sales
    else:
        ergebnis=None
    return ergebnis  

In [304]:
data_2_clean['Stückgebäck']=data_2_clean.apply(lambda row: def_sales(row['Stückgebäck'],row['Outlier']), axis=1)

### Weihnachtsartikel

In [347]:
data_2_clean.Weihnachtsartikel.describe(percentiles=[0.9, 0.99])

count    1979.000000
mean       21.198583
std        53.875080
min         0.000000
50%         0.000000
90%       125.578646
99%       203.102616
max       245.976731
Name: Weihnachtsartikel, dtype: float64

In [353]:
data_2_clean.fillna(method='bfill',inplace=True)

In [ ]:
# checking null values
nan = pd.DataFrame(data_2_clean.isnull().sum(),columns=['Count'])
#nan

# Export to csv

In [349]:
df_new_1.to_csv("df_new_1.csv")
df_new_3.to_csv("df_new_3.csv")
df_new.to_csv("df_new.csv")

In [354]:
df_new_2.to_csv("df_new_2.csv")
data_2_clean.to_csv("data_2_clean.csv")

In [ ]:
df_import_2=df_new_2.copy()

In [348]:
data_2_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1979 entries, 0 to 1978
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Datum               1979 non-null   datetime64[ns]
 1   Weizenbrot          1959 non-null   float64       
 2   Mischbrot           1959 non-null   float64       
 3   Vollkornbrot        1959 non-null   float64       
 4   Stangenbrote        1959 non-null   float64       
 5   Spezialbrot         1975 non-null   float64       
 6   Brötchen            1959 non-null   float64       
 7   Süsse_Brötchen      1959 non-null   float64       
 8   Herzhafte_Brötchen  1959 non-null   float64       
 9   KonditoreiBlech     1959 non-null   float64       
 10  Stückgebäck         1979 non-null   float64       
 11  Blechkuchen         1959 non-null   float64       
 12  Weihnachtsartikel   1979 non-null   float64       
 13  Wochentag           1979 non-null   int64       

In [153]:
article_selection=df_import_2.iloc[:,0:13].columns.tolist()

In [155]:
df_import_2.to_csv("df_import_2.csv")